# Task 1.6

In [1]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt
import scipy
import re

In [2]:
import spacy
from spacy import displacy

In [3]:
# Download English module

!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

## Load 20th Century Scrape

In [5]:
# Load the book

with open('20th_Century_Scrape.txt', 'r', errors='ignore') as file:
    data = file.read().replace('\n', '')

In [6]:
book = NER(data)

In [7]:
# Visualize identified entities

displacy.render(book[273:20000], style = "ent", jupyter = True)

## Get Named Entity List per Sentence

In [8]:
df_sentences = [] # empty shell to store results

# Loop through sentences, get entity list for each sentence
for sent in book.sents:
    entity_list = [ent.text for ent in sent.ents]
    df_sentences.append({"sentence": sent, "entities": entity_list})
    
df_sentences = pd.DataFrame(df_sentences)

In [9]:
df_sentences.head(10)

,sentence,entities
0,"(Key, events, of, the, 20th, century, -, Wikip...","[the 20th century - WikipediaJump, Navigation\..."
1,"(the, wars1.2.1Economic)",[]
2,"(depression1.2.2The, rise, of, dictatorship1.3...",[]
3,"(World, War, II, (, 1939–1945)1.3.1The, war, i...",[World War II]
4,"(days1.3.7The, war, in, the, Pacific1.3.7.1Bac...","[Pacific1.3.7.1Background1.3.8Japanese, Holoca..."
5,"(decolonization1.4.2The, Cold, War, (, 1947–19...","[decolonization1.4.2The Cold War, 1947–1991)1...."
6,"(The, World, Wars, sparked, tension, between, ...","[the Cold War, the Space Race, the World Wide ..."
7,"(These, advancements, have, played, a, signifi...","[the 21st century, today]"
8,"(Historic, events, in, the, 20th, century[edit...","[the 20th, Edwardian, the 20th century]"
9,"(The, 1900s, saw, the, decade, herald, a, seri...","[The 1900s, the decade]"


## Import and Clean Country List CSV

In [10]:
# Import countries list

countries_df = pd.read_csv("countries_list_20th_century_1.5.csv", index_col = 0)

In [11]:
countries_df.head()

,country_name
1,Afghanistan
2,Albania
3,Algeria
4,Andorra
5,Angola


In [12]:
#Remove unwanted spaces
countries_df['country_name'] = countries_df['country_name'].str.strip()

In [13]:
countries_df.head()

,country_name
1,Afghanistan
2,Albania
3,Algeria
4,Andorra
5,Angola


## Filtering Entities from Text

In [14]:
# Function to filter out entities not of interest

def filter_entity(ent_list, countries_df):
    return [ent for ent in ent_list 
            if ent in list(countries_df['country_name'])]

In [15]:
# Check

filter_entity(["Afghanistan","Germany", "Alice", "CF", "2"], countries_df)

['Afghanistan', 'Germany']

In [16]:
df_sentences['country_entities'] = df_sentences['entities'].apply(lambda x: filter_entity(x, countries_df))

In [17]:
df_sentences['country_entities'].head()

0    []
1    []
2    []
3    []
4    []
Name: country_entities, dtype: object

In [20]:
# Filter out sentences that don't have any character entities

df_sentences_filtered = df_sentences[df_sentences['country_entities'].map(len) > 0]

In [21]:
df_sentences_filtered.tail(10)

,sentence,entities,country_entities
824,"(Past, &, Present, ., 210, :, 103–120, ., doi:...","[Past & Present, 210, 103–120, Germany, The Co...",[Germany]
845,"("", The, forgotten, violence, that, helped, In...",[India],[India]
848,"("", Indian, Independence, Day, :, everything, ...","[Indian Independence Day, Partition, India, Pa...","[India, Pakistan]"
854,"(Retrieved, 2018, -, 12, -, 18.^, "", The, Phil...","[2018-12-18.^, Philippines, 1898–1946, US Hous...",[Philippines]
901,"(The, Moldovans, :, Romania, ,, Russia, ,, and...","[Moldovans, Romania, Russia, the Politics of C...","[Romania, Russia]"
947,"("", Selling, "", Operation, Passage, to, Freedo...","[Thomas Dooley, the Religious Overtones of Ear...",[Vietnam]
970,"("", Stuck, in, Endless, Preliminaries, :, Viet...","[Vietnam, the Battle of the Paris Peace Table,...",[Vietnam]
1145,"("", Anti, -, American, Behavior, in, the, Midd...","[Anti-American, the Middle East, Lebanon]",[Lebanon]
1148,"(The, Rise, of, China, and, India, :, A, New, ...","[China, India, New Asian Drama]",[India]
1149,"(Singapore, :, World, Scientific, .)","[Singapore, World Scientific]",[Singapore]


In [22]:
df_sentences_filtered.shape

(121, 3)

## Relationships

In [23]:
# Defining relationships 

# window size = 5 : this defines how many sentences will be looked at simultaneously 
relationships = [] # create an empty list

for i in range(df_sentences_filtered.index[-1]):
    end_i = min(i+5, df_sentences_filtered.index[-1])
    country_list = sum((df_sentences_filtered.loc[i: end_i].country_entities), [])
    
    # Remove duplicated characters that are next to each other
    country_unique = [country_list[i] for i in range(len(country_list)) 
                   if (i==0) or country_list[i] != country_list[i-1]]
    
    if len(country_unique) > 1:
        for idx, a in enumerate(country_unique[:-1]):
            b = country_unique[idx + 1]
            relationships.append({"source": a, "target": b})

In [24]:
relationship_df = pd.DataFrame(relationships)

In [25]:
relationship_df

,source,target
0,France,Italy
1,Italy,Russia
2,France,Italy
3,Italy,Russia
4,Russia,Germany
...,...,...
612,Lebanon,India
613,India,Singapore
614,India,Singapore
615,India,Singapore


In [26]:
# Sort the cases with a->b and b->a

relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df.head(5)

,source,target
0,France,Italy
1,Italy,Russia
2,France,Italy
3,Italy,Russia
4,Germany,Russia


In [27]:
# Summarize the interactions

relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source","target"], sort=False, as_index=False).sum()

In [28]:
relationship_df.head(10)

,source,target,value
0,France,Italy,10
1,Italy,Russia,6
2,Germany,Russia,26
3,Bulgaria,Germany,6
4,Bulgaria,Russia,6
5,Germany,Italy,26
6,Austria,Germany,11
7,Germany,Spain,1
8,France,Poland,11
9,France,Germany,31


In [29]:
relationship_df.to_csv('countries_relationship.csv')